In [1]:
# import stuff here
import requests
import pymongo
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd


In [2]:
# Set up splinter
executable_path = {'executable_path':ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Sharon\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [3]:
# NASA Mars News ....
# Use the splinter browser... this cell took waaayyy longer than it should have.
nasa_url = "https://redplanetscience.com/"

browser.visit(nasa_url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

results = soup.find_all('div', class_='list_text')

results

[<div class="list_text">
 <div class="list_date">April 10, 2021</div>
 <div class="content_title">NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars</div>
 <div class="article_teaser_body">For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">April 9, 2021</div>
 <div class="content_title">NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light</div>
 <div class="article_teaser_body">Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.</div>
 </div>,
 <div class="list_text">
 <div class="list_date">April 8, 2021</div>
 <div class="content_title">How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus </div>
 <div class="artic

In [4]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.nasa_db
collection = db.items



In [5]:
# Loop through results and populate Mongo nasa_db
for result in results:
    try:
        title = result.find('div', class_='content_title').text
        
        blurb = result.find('div', class_='article_teaser_body').text
        
        print('**************')
        print(f'Title: {title}')
        print(f'Blurb: {blurb}')
        
        # gather up the items for mongo db
        if (title and blurb):
           post = {
               'title': title,
               'blurb': blurb,
            }

           collection.insert_one(post)
    except Exception as e:
        print(e)
    

**************
Title: NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars
Blurb: For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design.
**************
Title: NASA's MAVEN Observes Martian Night Sky Pulsing in Ultraviolet Light
Blurb: Vast areas of the Martian night sky pulse in ultraviolet light, according to images from NASA’s MAVEN spacecraft. The results are being used to illuminate complex circulation patterns in the Martian atmosphere.
**************
Title: How NASA's Perseverance Mars Team Adjusted to Work in the Time of Coronavirus 
Blurb: Like much of the rest of the world, the Mars rover team is pushing forward with its mission-critical work while putting the health and safety of their colleagues and community first.
**************
Title: NASA's Mars 2020 Rover Completes Its First Drive
Blurb: In a 10-plus-hour marathon, the rover steered, turned and drove in 3-foot (1-m

In [6]:
# JPL Mars Space Images - Featured Image

jpl_url = "https://spaceimages-mars.com/"

browser.visit(jpl_url)

html = browser.html
jpl_soup = BeautifulSoup(html, 'html.parser')
jpl_soup


<html class=""><head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://maxcdn.bootstrapcdn.com/bootstrap/4.0.0/css/bootstrap.min.css" rel="stylesheet"/>
<!-- <link rel="stylesheet" type="text/css" href="css/font.css"> -->
<link href="css/app.css" rel="stylesheet" type="text/css"/>
<link href="https://stackpath.bootstrapcdn.com/font-awesome/4.7.0/css/font-awesome.min.css" rel="stylesheet" type="text/css"/>
<title>Space Image</title>
<style type="text/css">.fancybox-margin{margin-right:17px;}</style></head>
<body>
<div class="header">
<nav class="navbar navbar-expand-lg">
<a class="navbar-brand" href="#"><img id="logo" src="image/nasa.png"/><span class="logo">Jet Propulsion Laboratory</span>
<span class="logo1">California Institute of Technology</span></a>
<button aria-controls="navbarNav" aria-expanded="false" aria-label="Toggle navigation" class="navbar-toggler" data-target="#navbarNav" data-toggle="collapse" type="button

In [7]:
#jpl_results = jpl_soup.find_all('div', class_='floating_text_area')
jpl_results = jpl_soup.find_all('img', class_='headerimage fade-in')

featured_image_url = f"{jpl_url}{jpl_results[0]['src']}"
featured_image_url

'https://spaceimages-mars.com/image/featured/mars2.jpg'

In [ ]:
# Mars Facts
mars_facts_url = 'https://galaxyfacts-mars.com/'

tables = pd.read_html(mars_facts_url)
#tables
mars_earth_table_df = tables[0]
mars_earth_table_df

In [ ]:
mars_profile_table_df = tables[1]

In [ ]:

#
me_list = mars_earth_table_df.loc[0]
mars_earth_table_df.columns = me_list

mars_earth_table_df

In [ ]:
print(mars_earth_table_df.to_html())

In [ ]:

mars_profile_table_df

In [ ]:
mp_list =["Mars Attribute", "Measurement"]
mars_profile_table_df.columns = mp_list

print(mars_profile_table_df.to_html())

In [ ]:
# Mars Hemispheres  - high resolutions images for each of Mars' hemispheres
hemi_base_url = "https://marshemispheres.com/images/"

# Valles Marineris
valles_marineris_url = f'{hemi_base_url}valles_marineris_enhanced.tif'

# Cerberus
cerberus_url = f'{hemi_base_url}cerberus_enhanced.tif'

# Schiaparelli 
schiaparelli_url = f'{hemi_base_url}schiaparelli_enhanced.tif'

# Syrtis Major
syrtis_major_url = f'{hemi_base_url}syrtis_major_enhanced.tif'


In [ ]:
hemisphere_image_urls = [
    {"title": "Valles Marineris Hemisphere", "img_url": valles_marineris_url},
    {"title": "Cerberus Hemisphere", "img_url": cerberus_url},
    {"title": "Schiaparelli Hemisphere", "img_url": schiaparelli_url},
    {"title": "Syrtis Major Hemisphere", "img_url": syrtis_major_url},
]

hemisphere_image_urls